In [1]:
import sys
sys.path.append('../data_generation/')
import random
import tensorflow as tf

import numpy as np
import utils

from tensorflow import keras
from tensorflow.keras import layers

2023-09-01 10:16:24.497394: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def encode_sequences(sequences: list[str], residue_map: dict[str, int] = {'A':0,'U':1,'C':2,'G':3,'I':4,'N':5}) -> list[str]:
    encoded_sequences = []
    for seq in sequences:
        seq_array = np.array([residue_map[i] for i in seq])
        # encoded_array = np.zeros((seq_array.size, seq_array.max()+1), dtype=int)
        encoded_array = np.zeros((seq_array.size, len(residue_map)), dtype=int)
        encoded_array[np.arange(seq_array.size), seq_array] = 1
        encoded_sequences += [encoded_array]
    return encoded_sequences

In [3]:
# Load fragments
fragments = utils.load('../data_generation/data/fragments/fragments_8_filtered.pickle')

# Shuffle fragment features and labels
fragment_features = encode_sequences([i.res_seq for i in fragments])
fragment_labels = [np.zeros(44) for i in fragments]
fragment_data = list(zip(fragment_features, fragment_labels))
random.shuffle(fragment_data)
fragment_features, fragment_labels = [i for (i,j) in fragment_data], [j for (i,j) in fragment_data]

# Split data
split_ratio = 1/3
split_idx = int(len(fragment_features)*split_ratio)
train_features, test_features = fragment_features[:split_idx], fragment_features[split_idx:]
train_labels, test_labels = fragment_labels[:split_idx], fragment_labels[split_idx:]

# Preprovess datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_features, test_labels))

BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

# Build model
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(8, 6)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(45)
])

# Compile model

METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'),
    #   keras.metrics.CategoricalAccuracy(name='accuracy'),
    #   keras.metrics.Precision(name='precision'),
    #   keras.metrics.Recall(name='recall'),
    #   keras.metrics.AUC(name='auc'),
    #   keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=METRICS)

# Fit model
model.fit(train_dataset, epochs=50)

# Evaluate model
model.evaluate(test_dataset)

2023-09-01 10:16:27.242995: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-01 10:16:27.247668: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/50


ValueError: in user code:

    File "/home/xinhex/anaconda3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/xinhex/anaconda3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/xinhex/anaconda3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/xinhex/anaconda3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1028, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/home/xinhex/anaconda3/envs/tensorflow/lib/python3.9/site-packages/keras/engine/training.py", line 1122, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "/home